<h1 align="Center">Retrieving PI Point Data</h1>
In this notebook, we will explore the various ways to retrieve PI Point data using python and the AF SDK. 

<hr>
We will start off by loading in the AF SDK. Check out the previous notebook "1. Starting with AF SDK and Python" if you dont understand what this code does!

In [1]:
# pythonnet is imported as "clr"
# this package allows you to import dotNet DLL's and run them within your python environment
import clr
import os
import sys

# Get the installation directory from the environment variable or fall back to the Windows
# default installation path
PIAF_SDK = os.getenv('PIHOME', 'C:\\Program Files\\PIPC')
PIAF_SDK += '\\AF\\PublicAssemblies\\4.0\\'

# check if the AF SDK default directories exist
if not os.path.isdir(PIAF_SDK):
    raise ImportError('PIAF SDK not found in {}, check installation'.format(PIAF_SDK))
    
# append them to the PATH environment variable
sys.path.append(PIAF_SDK)

# add the AF SDK DLL 
clr.AddReference('OSIsoft.AFSDK')

# import the AF SDK
from OSIsoft import AF

print(AF)

<module 'OSIsoft.AF'>


In [2]:
# get first item in GetPIServers object
piserver = AF.PI.PIServers.GetPIServers()[0]

# search for pi point on the pi data archive
pipoint = AF.PI.PIPoint.FindPIPoint(piserver, "sinusoid")

print("Found {} on data archive {}".format(pipoint.Name, piserver.Name))

Found SINUSOID on data archive pi-pd1


<h3>PI Point Attributes</h3>
Let examine some properties of the PIPoint object.

In [3]:
# print immutable properties
print(pipoint.ID)
print(pipoint.PointClass)
print(pipoint.PointType)
print(pipoint.Server)
print(pipoint.Step)

1
classic
12
pi-pd1
False


You will notice there are many "properties" missing *(e.g. PointSource, InsturmentTag, Location1, Location2, ...)*. The PIPoint object actually does not consider those properties, but instead, <em><b>attributes</b></em>. The PIPoint Attributes list can be found using the <b>FindAttributeNames</b> method.

In [4]:
pipoint.FindAttributeNames("*")

['tag', 'pointtype', 'pointid', 'step', 'future', 'ptclassname']

<hr>
<h3>Retrieving PIPoint Values</h3>

In [5]:
interpolated_value = pipoint.InterpolatedValue('*-1h')

TypeError: No method matches given arguments for InterpolatedValue: (<class 'str'>)

Hmmm that didn't seem to work... This sh\*t is broken! Let's take a breath and read the error message.
<blockquote><font color="maroon"><b>TypeError</b></font>: No method matches given arguments for InterpolatedValue</blockquote>
The TypeError specifies that we have an incorrect object Type somewhere.. and the error message tells us exactly where. The <a href="https://techsupport.osisoft.com/Documentation/PI-AF-SDK/html/M_OSIsoft_AF_PI_PIPoint_InterpolatedValue.htm">InterpolatedValue</a> method does not have a parameter set that matches a String object as input.
<img src="./img/interpolated_value.png" />
<hr>
Looking at the documentation, it specifies the input object must be an AFTime object. Even more, it provides the namespace of the object <em>AF.Time.AFTime</em>. Let's try casting our time string into an AFTime object.

<hr>
<h3>PIPoint.InterpolatedValue Method</a>

In [6]:
interpolated_value = pipoint.InterpolatedValue(AF.Time.AFTime('*-1h'))
print(interpolated_value)

46.7014


Success! Let's take a look at the returned <em>interpolated_value</em> variable.

In [7]:
type(interpolated_value)

OSIsoft.AF.Asset.AFValue

Interestingly, it is not a float or string data type, but actually an AFValue object!
<hr>
Let's investigate what type of properties this object has within python. As always, a list of all the properties can be found in the AF SDK documentation for <a href="https://techsupport.osisoft.com/Documentation/PI-AF-SDK/Html/T_OSIsoft_AF_Asset_AFValue.htm">AFValue</a>.

In [8]:
print(interpolated_value.Timestamp)
print(interpolated_value.Attribute)
print(interpolated_value.IsGood)
print(interpolated_value.UOM)
print(interpolated_value.Value)
print(interpolated_value.ValueType)

4/21/2022 6:07:33 AM
None
True
None
46.7014
System.Single


<hr>
<h3>PIPoint.RecordedValue Method</h3>
Let's try a different method to retrieve PI data, the <a href="https://techsupport.osisoft.com/Documentation/PI-AF-SDK/html/M_OSIsoft_AF_PI_PIPoint_RecordedValue.htm">RecordedValue</a> method. This method will return *only* recorded values.

In [9]:
recorded_value = pipoint.RecordedValue(AF.Time.AFTime('*-1h'))
print("Timestamp: {} | Value: {}".format(recorded_value.Timestamp, recorded_value.Value))

Timestamp: 4/21/2022 6:07:44 AM | Value: 46.6256


Notice that both of the above methods return only a *single* value. What if we want to return many values of a PI Point?
To do this, we will use the InterpolatedValues, RecordedValues, and PlotValues methods.
<hr>
<h3>PIPoint.InterpolatedValues Method</h3>
The Interpolated Values method will yield a list of interpolated values over a given timespan. For this method, we also need to specify an AFTimeSpan object.

The timespan can be initialized with the following syntax.
<blockquote><font color="blue">public</font> AFTimeSpan(<br>
	&emsp;&emsp;int <em>years = 0</em>,<br>
	&emsp;&emsp;int <em>months = 0</em>,<br>
	&emsp;&emsp;int <em>days = 0</em>,<br>
	&emsp;&emsp;double <em>hours = 0</em>,<br>
	&emsp;&emsp;double <em>minutes = 0</em>,<br>
	&emsp;&emsp;double <em>seconds = 0</em>,<br>
	&emsp;&emsp;double <em>milliseconds = 0</em><br>
)<br></blockquote>
<hr>

Finally, we can retrieve the values using the InterpolatedValues method with the following syntax.
<blockquote><font color="blue">public</font> AFValues InterpolatedValues(<br>
    &emsp;&emsp;AFTimeRange <em>timeRange</em>,<br>
	&emsp;&emsp;AFTimeSpan <em>timeSpan</em>,<br>
	&emsp;&emsp;string <em>filterExpression</em>,<br>
	&emsp;&emsp;bool <em>includeFilteredValues</em><br>
)<br></blockquote>

In [12]:
# Define AFTimeSpan
five_minute_timespan = AF.Time.AFTimeSpan(0, 0, 0, 0.0, 5.0)

# Get Interpolated Values
interpolated_values = pipoint.InterpolatedValues(
    AF.Time.AFTimeRange('*-1h','*'), five_minute_timespan, '', True)
print([val.Value for val in interpolated_values])

[42.4836, 40.33458, 38.20395, 36.09578, 34.01407, 31.9628, 29.94586, 27.96709, 26.03027, 24.13907, 22.2971, 20.50786, 18.81752]


<hr>
<h3>PIPoint.RecordedValues Method</h3>
The RecordedValues method requires five parameters, described below in C# code. We must instantiate objects of each type to execute the method.
<blockquote><font color="blue">public</font> AFValues RecordedValues(<br>
    &emsp;&emsp;AFTimeRange <em>timeRange</em>,<br>
	&emsp;&emsp;AFBoundaryType <em>boundaryType</em>,<br>
	&emsp;&emsp;string <em>filterExpression</em>,<br>
	&emsp;&emsp;bool <em>includeFilteredValues</em>,<br>
	&emsp;&emsp;int <em>maxCount</em> = 0<br>
)<br></blockquote>

<hr>
In this case, we must define a new object type call AFBoundaryType. This tells PI how to retrieve values on the time boundaries of the query. The options are as follow.
<table>
<tr>
<th>Member name</th><th>Value</th><th>Description</th>
</tr>
<tr>
<td>Inside</td><td>0</td><td>Specifies to return the recorded values on the inside of the requested time range as the first and last values.</td>
</tr>
<tr>
<td>Outsdie</td><td>1</td><td>Specifies to return the recorded values on the outside of the requested time range as the first and last values.</td>
</tr>
<tr>
<td>Interpolated</td><td>2</td><td>Specifies to create an interpolated value at the end points of the requested time range if a recorded value does not exist at that time.</td>
</tr>
</table>


In [15]:
recorded_values = pipoint.RecordedValues(
    AF.Time.AFTimeRange('*-1h','*'), AF.Data.AFBoundaryType.Outside, '', True, 100)
print([val.Value for val in recorded_values])

[41.03141, 40.81687, 40.6025, 40.38832, 40.17431, 39.96049, 39.74687, 39.53344, 39.32021, 39.10718, 38.89436, 38.68174, 38.46935, 38.25718, 38.04522, 37.8335, 37.62201, 37.41075, 37.19973, 36.98896, 36.77843, 36.56816, 36.35814, 36.14838, 35.93889, 35.72966, 35.52071, 35.31203, 35.10363, 34.89551, 34.68768, 34.48015, 34.27291, 34.06596, 33.85933, 33.65299, 33.44697, 33.24127, 33.03588, 32.83082, 32.62608, 32.42168, 32.21761, 32.01387, 31.81048, 31.60744, 31.39799, 31.20241, 31.00043, 30.79881, 30.59755, 30.39667, 30.19615, 29.99602, 29.79627, 29.5969, 29.39792, 29.19933, 29.00113, 28.80334, 28.60595, 28.40897, 28.2124, 28.01624, 27.82051, 27.62519, 27.4303, 27.23584, 27.04182, 26.84823, 26.65508, 26.46238, 26.27012, 26.07832, 25.88697, 25.69608, 25.50565, 25.31569, 25.1262, 24.93719, 24.74865, 24.56059, 24.37301, 24.18593, 23.99933, 23.81323, 23.62763, 23.44253, 23.25794, 23.07385, 22.89028, 22.70722, 22.52469, 22.34267, 22.16119, 21.98023, 21.79981, 21.61992, 21.44057, 21.26177]


<hr>
<h3>PIPoint.PlotValues Method</h3>
The last to query PI data is the Plot Values method. This method returns the data required to render a plot from the data. In this case, only a time range and number of intervals are provided. Typically, the number of intervals would be the number of horizontal pixels in the trend.

<blockquote><font color="blue">public</font> AFValues PlotValues (<br>
    &emsp;&emsp;AFTimeRange <em>timeRange</em>,<br>
	&emsp;&emsp;int <em>intervals</em><br>
)<br></blockquote>

In [33]:
plot_values = pipoint.PlotValues(
    AF.Time.AFTimeRange('*-1d','*'), 20)
print([val.Value for val in plot_values])

[9.986672, 0.006402232, 0.003386993, 5.258548e-05, 9.084019, 9.209802, 33.79012, 33.99666, 64.68784, 64.89624, 89.97531, 90.10597, 99.9936, 99.99661, 99.99995, 90.91606, 90.79028, 66.21001, 66.00347, 35.31229, 35.10389, 10.02477, 9.894111, 0.006406631, 0.003390193, 5.218762e-05, 9.083861, 9.209642, 33.78986, 33.9964, 64.6881, 64.8965, 89.97547, 90.10613, 99.9936, 99.99661, 99.99995, 90.9159, 90.79012, 66.20975, 66.00321, 35.31203, 35.10363, 10.02461, <OSIsoft.AF.Asset.AFEnumerationValue object at 0x00000165E00785B0>]


We see the last value returned was not a number, but an AFEnumerationValue object. If we use the AFEnumerationValue.ToString method, we can return the value as a string.

In [35]:
last_value = list(plot_values)[-1]
print(last_value.Value.ToString())


No Data


In this case, the SDK returned a "No Data" enumeration value. When using many plotting tools, it will be important to filter these enumerated values as most plotting libraries will not understand how to plot non-numeric values.

In [36]:
# check type using isinstance
print(isinstance(last_value.Value, AF.Asset.AFEnumerationValue))

# create a filter of plot_values that checks if the object is of type AF.Asset.AFValue
filtered_plot_values = list(filter(lambda x: not isinstance(x.Value, AF.Asset.AFEnumerationValue), plot_values))
print([val.Value for val in filtered_plot_values])

True
[9.986672, 0.006402232, 0.003386993, 5.258548e-05, 9.084019, 9.209802, 33.79012, 33.99666, 64.68784, 64.89624, 89.97531, 90.10597, 99.9936, 99.99661, 99.99995, 90.91606, 90.79028, 66.21001, 66.00347, 35.31229, 35.10389, 10.02477, 9.894111, 0.006406631, 0.003390193, 5.218762e-05, 9.083861, 9.209642, 33.78986, 33.9964, 64.6881, 64.8965, 89.97547, 90.10613, 99.9936, 99.99661, 99.99995, 90.9159, 90.79012, 66.20975, 66.00321, 35.31203, 35.10363, 10.02461]


In this tutorial, we learned how to query multiple values from a single PI Point. In the following tutorials, we will create PI Point Searches and Query Multiple Values from PI Point Lists.